In [53]:
#default import 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from math import sqrt
# learners
from sklearn.datasets import make_blobs
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
#
from sklearn.model_selection import train_test_split 


In [69]:

# 使用するモデルの定義
def get_models():
    models = list()
    models.append(GradientBoostingClassifier())
    models.append(LogisticRegression(solver='liblinear'))
#     models.append(DecisionTreeClassifier())
    models.append(SVC(gamma='scale', probability=True))
    models.append(GaussianNB())
    models.append(KNeighborsClassifier())
    models.append(AdaBoostClassifier())
    models.append(BaggingClassifier(n_estimators=10))
    models.append(RandomForestClassifier(n_estimators=10))
    models.append(ExtraTreesClassifier(n_estimators=10))
    return models


# super learner に用いる特徴量の作成
def get_out_of_fold_predictions(X, y, models):
    meta_X, meta_y = list(),list()
    kfold = KFold(n_splits=10,shuffle =True)
    for train_ix,test_ix in kfold.split(X):
        
        fold_yhats = list()

        train_X, test_X = X[train_ix],X[test_ix]
        train_y, test_y = y[train_ix],y[test_ix]
        meta_y.extend(test_y)#ここよくわからん
#         print(len(test_X))
        
        for model in models:
            model.fit(train_X,train_y)
            yhat = model.predict(test_X)
            
            fold_yhats.append(yhat.reshape(len(yhat),1))
        meta_X.append(np.hstack(fold_yhats))
    return np.vstack(meta_X),np.asarray(meta_y)

# 特徴量として使うモデルの集合
def fit_base_models(X, y, models):
    for model in models:
        model.fit(X, y)
# 最終的に予測するモデル
def fit_meta_model(X, y):
    model = LogisticRegression(solver = 'liblinear')
#     model = GradientBoostingClassifier()
    model.fit(X, y)
    return model

# 評価用関数
def evaluate_models(X, y, models):
    for model in models:
        yhat = model.predict(X)
        mse = mean_squared_error(y, yhat)
        print('%s: RMSE %.3f' % (model.__class__.__name__, sqrt(mse)))
# super learner による予測
def super_learner_predictions(X,models,meta_model):
    meta_X = list()
    for model in models:
        yhat = model.predict(X)
        meta_X.append(yhat.reshape(len(yhat),1))
    meta_X = np.hstack(meta_X)
    
    return meta_model.predict(meta_X)

In [70]:
data = pd.read_csv('../data/chspred.csv')

X = data.drop('mi',axis = 1)
y = data['mi']

X,X_val,y,y_val = train_test_split(X,y,test_size = 0.5,random_state = 33)

models = get_models()

meta_X,meta_y = get_out_of_fold_predictions(X.to_numpy(),y.to_numpy(),models)

fit_base_models(X, y, models)

meta_model = fit_meta_model(meta_X, meta_y)

evaluate_models(X_val,y_val,models)
yhat = super_learner_predictions(X_val, models, meta_model)
print('Super Learner: RMSE %.3f' % (sqrt(mean_squared_error(y_val, yhat))))

GradientBoostingClassifier: RMSE 0.187
LogisticRegression: RMSE 0.170
SVC: RMSE 0.171
GaussianNB: RMSE 0.240
KNeighborsClassifier: RMSE 0.172
AdaBoostClassifier: RMSE 0.189
BaggingClassifier: RMSE 0.175
RandomForestClassifier: RMSE 0.171
ExtraTreesClassifier: RMSE 0.170
Super Learner: RMSE 0.164


In [58]:
yhat

array([0, 0, 0, ..., 0, 0, 0])